In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import scipy as sp

from textblob import TextBlob, Word




In [ ]:
import os
import sys

os.environ['SPARK_HOME'] = "C:\Users\Pedro\Downloads\spark-2.0.2-bin-hadoop2.7\spark-2.0.2-bin-hadoop2.7"


sys.path.append("C:\Users\Pedro\Downloads\spark-2.0.2-bin-hadoop2.7\spark-2.0.2-bin-hadoop2.7\python")
sys.path.append("C:\Users\Pedro\Downloads\spark-2.0.2-bin-hadoop2.7\spark-2.0.2-bin-hadoop2.7\python\lib\py4j-0.8.2.1-src.zip")

from spark_sklearn import GridSearchCV

In [2]:
movies = pd.read_pickle('movies.pkl')

#Replace missing values with actual Python NaN.
movies = movies.replace('N/A', np.NaN)
movies = movies.replace('NA', np.NaN)
movies = movies.replace('NULL', np.NaN)

#Set the imdbRating column as float instead of strings
movies.imdbRating = movies.imdbRating.astype(float)


In [3]:
#Makes a new Data Frame without movies with script missing.
movie_scripts = movies[(movies.script.notnull())].copy()
movie_scripts.reset_index(inplace = True)
del movie_scripts['index']

In [4]:
def sentiment(script, pages = 100, ma = 5):
    #Creates a moving average of the sentiment polarity of the script pages (or %)
    try:
        sentiment = []
        for i in range(pages+1):
            sentiment.append(TextBlob(script[i*len(script)/pages:(i+1)*len(script)/pages]).sentiment.polarity)
        MA = []
        for i in range(len(sentiment)-ma):
            MA.append(np.mean(sentiment[i:i+ma]))
        return MA
    except:
        return [0]
movie_scripts['sentiment'] = movies.script.apply(lambda x: sentiment(x))


In [5]:
movie_scripts['sentiment_avg'] = movie_scripts.sentiment.apply(lambda x: np.mean(x))
movie_scripts['sentiment_stdev'] = movie_scripts.sentiment.apply(lambda x: np.std(x))

In [6]:
#Remove rows where scripts were empty.
movie_scripts = movie_scripts[movie_scripts.sentiment_stdev != 0].copy()
del movie_scripts['level_0']
del movie_scripts['index']


KeyError: 'level_0'

In [7]:
import ast
#Select only the movies that were found with the IMDB API
movie_scripts = movie_scripts[movie_scripts.IMDB_info.apply(lambda x: ast.literal_eval(x['Response']))]


In [8]:
def awards(award):
    try:
        if 'Oscar' in award:
            return 1
        else:
            return 0
    except:
        return 0
movie_scripts.Awards = movie_scripts.Awards.apply(lambda x: awards(x)) 


C:\Anaconda2\lib\site-packages\pandas\core\generic.py:2701: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [9]:
movie_ratings = movie_scripts[['title','script','Actors','Awards','Director', 'Genre','Runtime', 'Writer','imdbRating', 'sentiment',  'sentiment_avg','sentiment_stdev']].copy()
movie_ratings = movie_ratings.dropna()
movie_ratings.reset_index(inplace = True, drop = True)

movie_revenue = movie_scripts[['title','script','Actors','Awards','Director', 'Genre', 'sentiment',  'sentiment_avg','sentiment_stdev', 'Revenue']].copy()
movie_revenue = movie_revenue.dropna()
movie_revenue.reset_index(inplace = True, drop = True)

movie_awards = movie_scripts[['title','script','Actors','Awards','Director', 'Genre', 'sentiment',  'sentiment_avg','sentiment_stdev']].copy()
movie_awards = movie_awards.dropna()
movie_awards.reset_index(inplace = True, drop = True)

movie_ratings.to_pickle('movie_ratings.pkl')
movie_revenue.to_pickle('movie_revenues.pkl')
movie_awards.to_pickle('movie_awards.pkl')